# Cleaning Agency Names and Locodes

* Cleaning Obligated Data using Crosswalks created in data_docs.ipynb

In [1]:
import pandas as pd
from siuba import *

import numpy as np

from datetime import date
from IPython.display import Markdown, HTML, display_html

from calitp import *

In [2]:
df = pd.read_csv('gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/clean_obligated_waiting.csv', low_memory=False).drop('Unnamed: 0', axis=1)

/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:73: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


## Makings of the Function:

In [3]:
cw1_locode =  pd.read_csv('gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/agencylocode_primary_crosswalk1.csv')

In [4]:
cw2_id = pd.read_csv('gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/null_locodes_crosswalk2.csv')

In [5]:
cw3_unmatched = pd.read_csv('gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/unmatched_estimate.csv')

In [6]:
df.head()

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,warning,projectID,projectNO,compare_id_locode
0,Obligated,BPMPL,5904(121),Humboldt County,2018-12-18,2018-12-18,2018-12-18,2018-12-18,2018-12-27,0.00,...,14 Bridges In Humboldt County,Bridge Preventive Maintenance - Deck Joints,3,NaN,NaN,NON-MPO,NaN,5904,121,True
1,Obligated,ER,32D0(008),Mendocino County,2018-12-17,2018-12-19,2018-12-20,2018-12-20,2018-12-27,11508.00,...,"Comptche Ukiah Road, Cr 223 Pm 17.25",Permanent Restoration,3,2018-12-17,2018-12-18,NON-MPO,NaN,32D0,8,False
2,Obligated,ER,4820(004),Humboldt County,2018-12-07,2018-12-21,2018-12-21,2018-12-21,2018-12-27,45499.64,...,Mattole Rd Pm 43.17,Permanent Restoration,5,2018-12-06,2018-12-07,NON-MPO,NaN,4820,4,False
3,Obligated,CML,5924(244),Sacramento County,2018-12-11,2018-12-11,2018-12-21,2018-12-27,2018-12-27,207002.00,...,Fair Oaks Blvd. Between Howe Ave And Munroe St,Create A Smart Growth Corridor With Barrier Se...,1,2018-12-07,2018-12-07,SACOG,NaN,5924,244,True
4,Obligated,CML,5924(214),Sacramento County,2018-12-05,2018-12-11,2018-12-21,2018-12-27,2018-12-27,0.00,...,Florin Rd Between Power Inn Rd. And Florin Per...,Streetscape (tc),3,2018-11-28,2018-12-04,SACOG,NaN,5924,214,True


In [7]:
def get_num(x):
    try:
        return int(x)
    except Exception:
        try:
            return float(x)
        except Exception:
            return x

In [8]:
df['locode'] = df['locode'].apply(get_num)
df['projectID']= df['projectID'].apply(get_num)

In [9]:
locode_map1 = dict(zip(cw1_locode['primary_agency_locode'], 
                          cw1_locode['primary_agency_name']))

In [10]:
locode_map1

{5002: 'Sacramento',
 5003: 'Benicia',
 5004: 'San Diego',
 5005: 'San Jose',
 5006: 'Los Angeles',
 5007: 'Santa Barbara',
 5008: 'Stockton',
 5009: 'Marysville',
 5010: 'Sonora',
 5012: 'Oakland',
 5013: 'Crescent City',
 5014: 'Alameda',
 5015: 'Placerville',
 5016: 'San Luis Obispo',
 5017: 'Eureka',
 5018: 'Nevada City',
 5019: 'Santa Clara',
 5020: 'Yreka City',
 5021: 'Arcata',
 5022: 'Petaluma',
 5023: 'Grass Valley',
 5024: 'Martinez',
 5025: 'Santa Cruz',
 5026: 'San Buenaventura',
 5027: 'Healdsburg',
 5028: 'Santa Rosa',
 5029: 'Redwood City',
 5030: 'Vallejo',
 5031: 'Watsonville',
 5032: 'Suisun City',
 5033: 'San Bernardino',
 5034: 'Gilroy',
 5036: 'Trinidad',
 5037: 'Chico',
 5038: 'Antioch',
 5039: 'Cloverdale',
 5041: 'San Leandro',
 5042: 'Napa',
 5043: 'San Rafael',
 5044: 'Visalia',
 5045: 'Salinas',
 5046: 'Woodland',
 5048: 'Wheatland',
 5049: 'Ukiah',
 5050: 'Hayward',
 5051: 'St Helena',
 5052: 'Red Bluff',
 5053: 'Livermore',
 5055: 'Anaheim',
 5056: 'Dixon',

In [26]:
df['primary_agency_name'] = df['locode'].map(locode_map1)

In [27]:
df.head()

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,seq,date_request_initiated,date_completed_request,mpo,warning,projectID,projectNO,compare_id_locode,primary_agency_name,primary_agency_name_2
0,Obligated,BPMPL,5904(121),Humboldt County,2018-12-18,2018-12-18,2018-12-18,2018-12-18,2018-12-27,0.00,...,3,NaN,NaN,NON-MPO,NaN,5904,121,True,Humboldt County,NaN
1,Obligated,ER,32D0(008),Mendocino County,2018-12-17,2018-12-19,2018-12-20,2018-12-20,2018-12-27,11508.00,...,3,2018-12-17,2018-12-18,NON-MPO,NaN,32D0,8,False,Mendocino County,NaN
2,Obligated,ER,4820(004),Humboldt County,2018-12-07,2018-12-21,2018-12-21,2018-12-21,2018-12-27,45499.64,...,5,2018-12-06,2018-12-07,NON-MPO,NaN,4820,4,False,Humboldt County,NaN
3,Obligated,CML,5924(244),Sacramento County,2018-12-11,2018-12-11,2018-12-21,2018-12-27,2018-12-27,207002.00,...,1,2018-12-07,2018-12-07,SACOG,NaN,5924,244,True,Sacramento County,NaN
4,Obligated,CML,5924(214),Sacramento County,2018-12-05,2018-12-11,2018-12-21,2018-12-27,2018-12-27,0.00,...,3,2018-11-28,2018-12-04,SACOG,NaN,5924,214,True,Sacramento County,NaN


In [13]:
locode_map2 = dict(zip(cw2_id['primary_agency_locode'], 
                          cw2_id['primary_agency_name']))

In [14]:
locode_map2

{5929: 'San Joaquin County',
 5006: 'Los Angeles',
 5008: 'Stockton',
 5938: 'Stanislaus County',
 5059: 'Modesto',
 5953: 'Los Angeles County',
 5165: 'Turlock',
 5939: 'Merced County',
 5085: 'Merced',
 6053: 'San Bernardino Associated Governments',
 5063: 'Santa Ana',
 5954: 'San Bernardino County',
 5241: 'Ceres',
 5385: 'Ridgecrest',
 5956: 'Riverside County',
 6066: 'San Diego Association of Governments',
 5932: 'Tuolumne County',
 5184: 'Tehachapi',
 5930: 'Calaveras County',
 5211: 'El Cajon',
 5129: 'Oxnard',
 5004: 'San Diego',
 5450: 'Santa Clarita',
 5958: 'Imperial County',
 5940: 'Mariposa County',
 5957: 'San Diego County',
 5282: 'Palm Springs',
 6071: 'Orange County Transportation Authority',
 5203: 'Chula Vista',
 5174: 'Holtville',
 5073: 'Orange',
 5955: 'Orange County',
 5381: 'San Marcos',
 5952: 'Ventura County',
 5065: 'Colton',
 5441: 'Moreno Valley',
 5255: 'Riverbank',
 5111: 'Whittier',
 5167: 'Brawley',
 5312: 'Costa Mesa',
 5948: 'Inyo County',
 5392: 'Tho

In [15]:
df['primary_agency_name_2'] = df['projectID'].map(locode_map2)

In [17]:
df.sample(10)

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,seq,date_request_initiated,date_completed_request,mpo,warning,projectID,projectNO,compare_id_locode,primary_agency_name,primary_agency_name_2
2797,Obligated,BRLO,5919(073),Placer County,2017-12-22,2017-12-22,2018-01-03,2018-01-03,2018-01-10,925299.00,...,6,2017-12-21,2017-12-22,SACOG,NaN,5919,73,True,Placer County,NaN
5478,Obligated,STP,5450(089),Santa Clarita,2016-10-07,2016-10-17,2016-10-31,2016-11-03,2016-11-08,0.00,...,2,NaN,NaN,SCAG,NaN,5450,89,True,Santa Clarita,Santa Clarita
7725,Obligated,SRTS,5446(016),Encinitas,2015-12-10,2015-12-10,2015-12-10,2015-12-10,2015-12-23,-42156.23,...,2,NaN,NaN,SDAG,NaN,5446,16,True,Encinitas,Encinitas
5150,Obligated,CML,5002(139),Sacramento,2016-12-19,2016-12-19,2016-12-20,2016-12-28,2017-01-06,0.00,...,3,NaN,NaN,SACOG,NaN,5002,139,True,Sacramento,NaN
19682,Obligated,STP,5957(091),San Diego County,2021-09-01,2021-09-07,2021-09-08,2021-09-20,2021-09-21,-51266.95,...,2,2021-09-01,2021-09-01,SDAG,NaN,5957,91,True,San Diego County,San Diego County
13834,Obligated,BRLS,5200(040),Burbank,2019-05-06,2019-05-06,2019-05-06,2019-05-06,2019-05-17,0.00,...,2,2019-05-01,2019-05-02,SCAG,NaN,5200,40,True,Burbank,Burbank
13668,Obligated,BPMPL,5060(307),Fresno,2019-04-25,2019-04-25,2019-04-25,2019-04-25,2019-05-03,0.00,...,2,2019-04-02,2019-04-04,CFCG,NaN,5060,307,True,Fresno,NaN
7370,Obligated,CML,5012(128),Oakland,2015-12-21,2016-02-03,2016-02-10,2016-02-10,2016-02-16,2352857.00,...,1,NaN,NaN,MTC,NaN,5012,128,True,Oakland,NaN
7255,Obligated,CML,5938(182),Stanislaus County,2016-02-25,2016-02-25,2016-02-25,2016-02-25,2016-03-01,-18390.18,...,4,NaN,NaN,STANCOG,NaN,5938,182,True,Stanislaus County,Stanislaus County
14349,Obligated,ER,36P0(006),Butte County,2019-07-10,2019-07-20,2019-08-09,2019-08-09,2019-08-12,110008.24,...,1,2019-07-09,2019-07-19,BCAG,NaN,36P0,6,False,Butte County,NaN


In [20]:
df['primary_agency_name'] = np.where(df.primary_agency_name.isnull(), 
                      df['primary_agency_name_2'],df['primary_agency_name'])
#df['primary_agency_name'] = np.where(df['primary_agency_name'] == 0, df['primary_agency_name_2'], df['primary_agency_name'])

In [21]:
df.sample(10)

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,seq,date_request_initiated,date_completed_request,mpo,warning,projectID,projectNO,compare_id_locode,primary_agency_name,primary_agency_name_2
12259,Obligated,STPLER,5006(316),Los Angeles,2014-04-03,2014-04-03,2014-04-03,2014-04-03,2014-04-04,0.00,...,4,NaN,NaN,SANDAG,NaN,5006,316,False,Los Angeles,Los Angeles
16677,Obligated,HSIP,5487(002),Jurupa Valley,2020-05-20,2020-06-09,2020-06-12,2020-06-12,2020-06-16,1034739.00,...,2,2020-05-13,2020-06-09,SCAG,NaN,5487,2,True,Jurupa Valley,NaN
8235,Obligated,HPLUL,5250(023),Lynwood,2015-08-14,2015-08-26,2015-08-27,2015-09-08,2015-09-14,2932760.00,...,1,NaN,NaN,SCAG,NaN,5250,23,True,Lynwood,NaN
13513,Obligated,BHLS,6003(051),"Golden Gate Bridge, Highway And Transportation...",2019-06-06,2019-06-10,2019-06-13,2019-06-13,2019-06-20,45239332.00,...,6,2019-05-28,2019-05-28,MTC,NaN,6003,51,True,"Golden Gate Bridge, Highway And Transportation...",NaN
5061,Obligated,STP,5114(017),Sonoma,2017-01-31,2017-01-31,2017-01-31,2017-01-31,2017-02-02,-16433.54,...,3,NaN,NaN,MTC,NaN,5114,17,True,Sonoma,NaN
4336,Obligated,CML,5060(215),Fresno,2017-05-11,2017-05-11,2017-05-11,2017-05-11,2017-05-19,-79645.12,...,5,NaN,NaN,CFCG,NaN,5060,215,True,Fresno,NaN
14510,Obligated,ACST-ER,32L0(230),Humboldt County,2019-08-22,2019-08-28,2019-08-30,2019-09-05,2019-09-09,0.00,...,4,2019-08-22,2019-08-26,NON-MPO,NaN,32L0,230,False,Humboldt County,NaN
17053,Obligated,CML,5919(128),Placer County,2020-09-04,2020-09-04,2020-09-09,2020-09-11,2020-09-15,1941758.00,...,5,2020-09-04,2020-09-04,SACOG,NaN,5919,128,True,Placer County,NaN
19925,FMIS,STP,6066(173),San Diego Association Of Governments,2021-09-16,2021-10-14,2021-10-25,2021-10-25,2021-10-25,0.00,...,2,2021-09-16,2021-09-16,SDAG,NaN,6066,173,True,San Diego Association of Governments,San Diego Association of Governments
12056,Obligated,BRLO,5905(082),Trinity County,2014-03-26,2014-03-26,2014-04-09,2014-04-22,2014-04-23,950000.00,...,2,NaN,NaN,NON-MPO,NaN,5905,82,True,Trinity County,NaN


In [22]:
len(df>>filter(_.primary_agency_name.isnull())>>filter(_.primary_agency_name_2.isnull()))

57

In [23]:
(df
 >>filter(_.primary_agency_name.isnull())
 >>filter(_.primary_agency_name_2.isnull())
 >>count(_.agency)
 >> arrange(-_.n)
)

,agency,n
6,Inyo,14
9,Lancaster,6
25,Yucaipa,3
1,Banning,2
3,Fowler,2
4,Gold Coast Transit District,2
8,La Quinta,2
10,Los Angeles,2
12,Palmdale,2
14,San Bernardino,2


In [24]:
#cw3_unmatched
## not running beacause of an error

In [25]:
# locode_map3 = dict(zip(cw3_unmatched['agency_locode'], 
#                           cw3_unmatched['primary_agency_name']))

In [26]:
# df['primary_agency_name_3'] = df['projectID'].map(locode_map3)
# df['primary_agency_name_4'] = df['locode'].map(locode_map3)

## Converting to a Function

In [52]:
#25 agencies in the unmatched df. 

In [54]:
#may need to change `agency_locode_id` column name to make this work 

In [10]:
def clean_agency_names(df):
    
    df =  pd.read_csv('gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/clean_obligated_waiting.csv', low_memory=False).drop('Unnamed: 0', axis=1)
    
    cw1_locode =  pd.read_csv('gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/agencylocode_primary_crosswalk1.csv')
    cw2_id = pd.read_csv('gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/null_locodes_crosswalk2.csv')
    
    # ensure the values in crosswalk are int64 
    
    # df['locode'] = df['locode'].apply(get_num)
    # df['projectID']= df['projectID'].apply(get_num)
    
    # create a dictionary for the data
    
    locode_map1 = dict(zip(cw1_locode['primary_agency_locode'], 
                          cw1_locode['primary_agency_name']))
    
    locode_map2 = dict(zip(cw2_id['primary_agency_locode'], 
                          cw2_id['primary_agency_name']))
    
    df['primary_agency_name'] = df['locode'].map(locode_map1)
    df['primary_agency_name2'] = df['projectID'].map(locode_map2)
    
    return df

In [11]:
df_clean = clean_agency_names(df)